# Transport Suite (Notebook)

แปลงจากสคริปต์ Python ให้รันบน Jupyter Notebook ได้

In [ ]:
# -*- coding: utf-8 -*-
"""
การรัน:
  python transport_suite.py --method เลือก(nwc, lcm, vam) --optimal

ตัวอย่าง:
  python transport_suite.py --method vam --optimal
  python transport_suite.py --method nwc --optimal
  python transport_suite.py --method lcm --optimal
"""
from __future__ import annotations



from typing import List, Tuple, Dict, Set
import argparse, math, collections

TOL = 1e-12

# -------------------- Helpers for 1-based display --------------------
def idx1(k: int) -> int:
    return k + 1

def cell1(i: int, j: int) -> str:
    return f"({i+1},{j+1})"

def fmt_loop(loop):
    return "[" + " -> ".join(cell1(i, j) for (i, j) in loop) + "]"

# -------------------- Utilities --------------------
def balance_transport(supply: List[float], demand: List[float], cost: List[List[float]]):
    """Balance problem by adding a dummy row/col if needed (cost=0)."""
    S, D = sum(supply), sum(demand)
    m, n = len(supply), len(demand)
    C = [row[:] for row in cost]
    added = {"dummy_row": False, "dummy_col": False}
    if abs(S - D) < 1e-9:
        return supply[:], demand[:], C, added
    if S < D:
        C.append([0.0] * n)
        supply = supply[:] + [D - S]
        added["dummy_row"] = True
    else:
        for r in C:
            r.append(0.0)
        demand = demand[:] + [S - D]
        added["dummy_col"] = True
    return supply, demand, C, added

def total_cost(plan, cost) -> float:
    m, n = len(plan), len(plan[0])
    return sum(plan[i][j] * cost[i][j] for i in range(m) for j in range(n))

def _fmt_num(x: float) -> str:
    s = f"{x:.2f}"
    return s.rstrip("0").rstrip(".") if "." in s else s

def print_transport_table(plan, cost, supply=None, demand=None, title="Plan (qty@unit_cost)", show_zeros=False):
    """
    แสดงตารางขนส่งสวย ๆ:
      - เซลล์เป็น 'จำนวน@ต้นทุน'
      - แสดงผลรวมแถว (RowSum) เทียบ Supply
      - แสดงผลรวมคอลัมน์ (ColSum) เทียบ Demand
    NOTE: ใช้ index แสดงผลแบบ 1-based
    """
    m, n = len(plan), len(plan[0])
    # คำนวณยอดรวม
    row_sum = [sum(plan[i][j] for j in range(n)) for i in range(m)]
    col_sum = [sum(plan[i][j] for i in range(m)) for j in range(n)]

    # ถ้าไม่มี supply/demand ที่บาลานซ์มา ให้ derive จากแผน
    if supply is None: supply = row_sum[:]
    if demand is None: demand = col_sum[:]

    colw = 16
    def sep():
        print("-" * (colw * (n + 3) + 1))  # +3 = label + RowSum + Supply

    print("\n" + title)
    sep()

    # header 1: ชื่อปลายทาง
    header = ["".ljust(colw)]
    for j in range(n):
        header.append(f"Dest{j+1}".center(colw))
    header += ["| RowSum".center(colw), "| Supply".center(colw)]
    print("|" + "|".join(header) + "|")

    # header 2: demand
    demand_line = ["Demand".ljust(colw)]
    for j in range(n):
        demand_line.append(_fmt_num(demand[j]).center(colw))
    demand_line += ["|".center(colw), "|".center(colw)]
    print("|" + "|".join(demand_line) + "|")
    sep()

    # body
    for i in range(m):
        row_cells = [f"Src{i+1}".ljust(colw)]
        for j in range(n):
            q = plan[i][j]
            txt = f"{_fmt_num(q)}@{_fmt_num(cost[i][j])}" if (q != 0 or show_zeros) else ""
            row_cells.append(txt.center(colw))
        row_cells += [f"| {_fmt_num(row_sum[i])}".center(colw), f"| {_fmt_num(supply[i])}".center(colw)]
        print("|" + "|".join(row_cells) + "|")
    sep()

    # footer: col sums
    colsum_line = ["ColSum".ljust(colw)]
    for j in range(n):
        colsum_line.append(_fmt_num(col_sum[j]).center(colw))
    colsum_line += ["|".center(colw), "|".center(colw)]
    print("|" + "|".join(colsum_line) + "|")
    sep()

# -------------------- Initial methods --------------------
def northwest_corner(supply: List[float], demand: List[float], cost: List[List[float]], verbose=True):
    supply, demand, cost, added = balance_transport(supply, demand, cost)
    m, n = len(supply), len(demand)
    plan = [[0.0 for _ in range(n)] for _ in range(m)]
    if verbose:
        print("=== Northwest Corner (NWC) ===")
        print("Balanced problem:")
        print(f"  supply = {supply}")
        print(f"  demand = {demand}")
        print(f"  added = {added}\n")
    i = j = 0
    step = 0
    while i < m and j < n:
        step += 1
        alloc = min(supply[i], demand[j])
        plan[i][j] = alloc
        supply[i] -= alloc; demand[j] -= alloc
        if verbose:
            print(f"Step {step}: allocate {alloc:.4f} at cell {cell1(i,j)}  cost={cost[i][j]:.4f}")
            print(f"  Remaining: supply[{idx1(i)}]={supply[i]:.4f}, demand[{idx1(j)}]={demand[j]:.4f}")
        if abs(supply[i]) < TOL and abs(demand[j]) < TOL:
            if verbose: print("  Both exhausted -> move diagonal (degenerate).")
            i += 1; j += 1
        elif abs(supply[i]) < TOL:
            if verbose: print("  Row exhausted -> move down.")
            i += 1
        elif abs(demand[j]) < TOL:
            if verbose: print("  Column exhausted -> move right.")
            j += 1
        if verbose: print()
    return plan, cost, added

def least_cost_method(supply: List[float], demand: List[float], cost: List[List[float]], verbose=True):
    supply, demand, cost, added = balance_transport(supply, demand, cost)
    m, n = len(supply), len(demand)
    plan = [[0.0 for _ in range(n)] for _ in range(m)]
    active_rows = {i for i in range(m) if supply[i] > 0}
    active_cols = {j for j in range(n) if demand[j] > 0}
    if verbose:
        print("=== Least Cost Method (LCM) ===")
        print("Balanced problem:")
        print(f"  supply = {supply}")
        print(f"  demand = {demand}")
        print(f"  added = {added}\n")
    step = 0
    while active_rows and active_cols:
        step += 1
        best = None
        for i in active_rows:
            for j in active_cols:
                cand = (cost[i][j], i, j)
                if best is None or cand < best:
                    best = cand
        cmin, i, j = best
        alloc = min(supply[i], demand[j])
        plan[i][j] += alloc
        supply[i] -= alloc; demand[j] -= alloc
        if verbose:
            print(f"Step {step}: choose cheapest cell {cell1(i,j)} with cost={cmin:.4f}")
            print(f"  Allocate {alloc:.4f}")
            print(f"  Remaining: supply[{idx1(i)}]={supply[i]:.4f}, demand[{idx1(j)}]={demand[j]:.4f}")
        if abs(supply[i]) < TOL:
            active_rows.discard(i)
            if verbose: print(f"  Row exhausted -> remove row {idx1(i)}")
        if abs(demand[j]) < TOL:
            active_cols.discard(j)
            if verbose: print(f"  Column exhausted -> remove column {idx1(j)}")
        if verbose: print()
    return plan, cost, added

def _two_smallest(values: List[float]) -> Tuple[float, float]:
    inf = float("inf"); a, b = inf, inf
    for v in values:
        if v < a: a, b = v, a
        elif v < b: b = v
    return a, b

def vogel_approximation(supply: List[float], demand: List[float], cost: List[List[float]], verbose=True):
    supply, demand, cost, added = balance_transport(supply, demand, cost)
    m, n = len(supply), len(demand)
    plan = [[0.0 for _ in range(n)] for _ in range(m)]
    active_rows = {i for i in range(m) if supply[i] > 0}
    active_cols = {j for j in range(n) if demand[j] > 0}
    if verbose:
        print("=== Vogel's Approximation Method (VAM) ===")
        print("Balanced problem:")
        print(f"  supply = {supply}")
        print(f"  demand = {demand}")
        print(f"  added = {added}\n")
    step = 0
    while active_rows and active_cols:
        step += 1
        # penalties
        rpen, cpen = {}, {}
        for i in active_rows:
            vals = [cost[i][j] for j in active_cols]
            m1, m2 = _two_smallest(vals); rpen[i] = (m1, m2, (m2 - m1) if m2 != float("inf") else m1)
        for j in active_cols:
            vals = [cost[i][j] for i in active_rows]
            m1, m2 = _two_smallest(vals); cpen[j] = (m1, m2, (m2 - m1) if m2 != float("inf") else m1)
        if verbose:
            print(f"Step {step}: penalties")
            for j in sorted(active_cols):
                m1,m2,p = cpen[j]; m2s = "∞" if m2 == float('inf') else f"{m2:.4f}"
                print(f"  col[{idx1(j)}] -> min1={m1:.4f}, min2={m2s}, penalty={p:.4f}")
            for i in sorted(active_rows):
                m1,m2,p = rpen[i]; m2s = "∞" if m2 == float('inf') else f"{m2:.4f}"
                print(f"  row[{idx1(i)}] -> min1={m1:.4f}, min2={m2s}, penalty={p:.4f}")
            print()
        # choose by max penalty; tie-break by smaller cheapest cell then prefer rows
        best = None  # (penalty, -cheapest, is_row, -idx)
        chosen_type, chosen_idx = None, None
        for i in active_rows:
            m1,m2,p = rpen[i]; score = (p, -m1, 1, -i)
            if best is None or score > best:
                best, chosen_type, chosen_idx = score, 'row', i
        for j in active_cols:
            m1,m2,p = cpen[j]; score = (p, -m1, 0, -j)
            if best is None or score > best:
                best, chosen_type, chosen_idx = score, 'col', j
        if chosen_type == 'row':
            i = chosen_idx
            j = min(active_cols, key=lambda jj: (cost[i][jj], jj))
        else:
            j = chosen_idx
            i = min(active_rows, key=lambda ii: (cost[ii][j], ii))
        alloc = min(supply[i], demand[j])
        plan[i][j] += alloc
        supply[i] -= alloc; demand[j] -= alloc
        if verbose:
            print(f"  Chosen: {chosen_type}[{idx1(chosen_idx)}] with max penalty {best[0]:.4f}")
            print(f"  Allocate {alloc:.4f} units to cell {cell1(i,j)} with cost {cost[i][j]:.4f}")
            print(f"  Remaining supply[{idx1(i)}]={supply[i]:.4f}, demand[{idx1(j)}]={demand[j]:.4f}\n")
        if abs(supply[i]) < TOL: active_rows.discard(i)
        if abs(demand[j]) < TOL: active_cols.discard(j)
    return plan, cost, added

# -------------------- Stepping-Stone improvement --------------------
def positives_as_basics(plan):
    basics = set()
    for i,row in enumerate(plan):
        for j,x in enumerate(row):
            if x > TOL: basics.add((i,j))
    return basics

class DSU:
    def __init__(self, n): self.p = list(range(n))
    def find(self, x):
        while self.p[x] != x:
            self.p[x] = self.p[self.p[x]]; x = self.p[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb: self.p[rb] = ra; return True
        return False

def ensure_spanning_tree(plan, cost, basic_zeros: Set[Tuple[int,int]]):
    m, n = len(plan), len(plan[0])
    N = m + n
    dsu = DSU(N)
    basics = positives_as_basics(plan).union(basic_zeros)
    edges = []
    for (i,j) in basics:
        if dsu.union(i, m+j):
            edges.append((i,j))
    while len(edges) < m + n - 1:
        candidate = None; best_cost = float('inf')
        for i in range(m):
            for j in range(n):
                if (i,j) in basics: continue
                if dsu.find(i) != dsu.find(m+j):
                    c = cost[i][j]
                    if c < best_cost: best_cost, candidate = c, (i,j)
        if candidate is None: break
        i,j = candidate
        basic_zeros.add((i,j)); basics.add((i,j)); dsu.union(i, m+j); edges.append((i,j))
    return basic_zeros

def build_basis_graph(plan, basic_zeros):
    m, n = len(plan), len(plan[0])
    basics = positives_as_basics(plan).union(basic_zeros)
    g = collections.defaultdict(list)
    for (i,j) in basics:
        g[('r',i)].append(('c',j))
        g[('c',j)].append(('r',i))
    return g

def path_row_to_col(g, i0, j0):
    start = ('r', i0); goal = ('c', j0)
    from collections import deque
    q = deque([start]); prev = {start: None}
    while q:
        u = q.popleft()
        if u == goal: break
        for v in g[u]:
            if v not in prev:
                prev[v] = u; q.append(v)
    if goal not in prev: return None
    nodes = []; cur = goal
    while cur is not None: nodes.append(cur); cur = prev[cur]
    nodes.reverse()
    cells = []
    for a,b in zip(nodes[:-1], nodes[1:]):
        if a[0]=='r' and b[0]=='c': cells.append((a[1], b[1]))
        elif a[0]=='c' and b[0]=='r': cells.append((b[1], a[1]))
        else: raise RuntimeError("Invalid alternation")
    return cells

def loop_for_cell(plan, basic_zeros, start):
    i0, j0 = start
    g = build_basis_graph(plan, basic_zeros)
    path_cells = path_row_to_col(g, i0, j0)
    if not path_cells: return None
    return [start] + path_cells + [start]

def compute_delta(cost, loop):
    delta = 0.0
    for k in range(len(loop)-1):  # ignore last repeated start
        i,j = loop[k]
        delta += cost[i][j] * (1 if k%2==0 else -1)
    return delta

def stepping_stone(plan, cost, verbose=True):
    """Improve 'plan' in-place until no negative Δ remains. Returns improved plan."""
    m, n = len(plan), len(plan[0])
    basic_zeros: Set[Tuple[int,int]] = set()
    basic_zeros = ensure_spanning_tree(plan, cost, basic_zeros)

    iter_no = 0
    while True:
        iter_no += 1
        best = None  # (delta, loop)
        if verbose:
            print(f"Iteration {iter_no}: evaluate Δ for all nonbasic cells")
        basics_now = positives_as_basics(plan).union(basic_zeros)
        for i in range(m):
            for j in range(n):
                if (i,j) in basics_now: continue
                loop = loop_for_cell(plan, basic_zeros, (i,j))
                if not loop: continue
                delta = compute_delta(cost, loop)
                if verbose:
                    print(f"  cell{cell1(i,j)} -> Δ={delta:.4f}  loop={fmt_loop(loop)}")
                if best is None or delta < best[0]:
                    best = (delta, loop)
        if best is None or best[0] >= -1e-12:
            if verbose: print("\nNo Δ < 0 -> current plan is optimal.\n")
            break
        delta, loop = best
        if verbose:
            print(f"\n=> Choose entering {cell1(*loop[0])} with most negative Δ={delta:.4f}")
        minus_cells = [loop[k] for k in range(1, len(loop)-1, 2)]
        theta = min(plan[i][j] for (i,j) in minus_cells)
        if verbose:
            print(f"   '-' positions: {[cell1(i,j) for (i,j) in minus_cells]} -> θ={theta:.4f}")
        # Update along loop
        for k in range(len(loop)-1):
            i,j = loop[k]
            if k%2==0:
                plan[i][j] += theta
            else:
                plan[i][j] -= theta
                if plan[i][j] < TOL: plan[i][j] = 0.0
        # Refresh zero-basics and ensure spanning tree again
        basic_zeros = {c for c in basic_zeros if c not in minus_cells}
        basic_zeros = ensure_spanning_tree(plan, cost, basic_zeros)
        if verbose:
            print(f"   New total cost = {total_cost(plan, cost):.4f}\n")
    return plan

# -------------------- Export CSV --------------------
def export_plan_to_csv(plan, cost, supply, demand, path="plan.csv"):
    import csv
    m, n = len(plan), len(plan[0])
    row_sum = [sum(plan[i][j] for j in range(n)) for i in range(m)]
    col_sum = [sum(plan[i][j] for i in range(m)) for j in range(n)]
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        # header
        w.writerow([""] + [f"Dest{j+1}" for j in range(n)] + ["RowSum", "Supply"])
        w.writerow(["Demand"] + [ _fmt_num(d) for d in demand ] + ["", ""])
        # rows
        for i in range(m):
            row = [f"Src{i+1}"]
            for j in range(n):
                q = plan[i][j]
                row.append(f"{_fmt_num(q)}@{_fmt_num(cost[i][j])}" if q != 0 else "")
            row += [_fmt_num(row_sum[i]), _fmt_num(supply[i])]
            w.writerow(row)
        # footer col sums
        w.writerow(["ColSum"] + [ _fmt_num(c) for c in col_sum ] + ["", ""])

# -------------------- Orchestrator --------------------
def solve_transport(supply, demand, cost, method: str, do_optimal: bool, quiet: bool):
    method = method.lower()
    verbose = not quiet
    if method == 'nwc':
        plan, cost_mat, added = northwest_corner(supply[:], demand[:], cost, verbose=verbose)
    elif method == 'lcm':
        plan, cost_mat, added = least_cost_method(supply[:], demand[:], cost, verbose=verbose)
    elif method == 'vam':
        plan, cost_mat, added = vogel_approximation(supply[:], demand[:], cost, verbose=verbose)
    else:
        raise ValueError("method must be one of: nwc, lcm, vam")

    # เอา supply/demand ที่บาลานซ์มาไว้โชว์ในตาราง (ให้มิติตรงกับ plan)
    supply_bal, demand_bal, _, _ = balance_transport(supply[:], demand[:], cost)

    print("\n=== Data ===")
    print("supply =", supply)
    print("demand =", demand)
    print("cost   =", cost)

    print_transport_table(plan, cost_mat, supply_bal, demand_bal,
                          title=f"{method.upper()} Initial Feasible Plan (qty@unit_cost)",
                          show_zeros=False)
    print(f"\nInitial total cost = {total_cost(plan, cost_mat):.4f}\n")
    if added["dummy_row"] or added["dummy_col"]:
        print("Note: Dummy row/column was added for balancing (cost=0).")

    # export initial
    export_plan_to_csv(plan, cost_mat, supply_bal, demand_bal, path="initial_plan.csv")
    print("Saved: initial_plan.csv")

    if not do_optimal:
        return

    print("\n=== Stepping-Stone Improvement ===")
    improved = stepping_stone(plan, cost_mat, verbose=verbose)
    print_transport_table(improved, cost_mat, supply_bal, demand_bal,
                          title="Optimal Plan (qty@unit_cost)",
                          show_zeros=False)
    print(f"\nTotal transportation cost = {total_cost(improved, cost_mat):.4f}")

    # export optimal
    export_plan_to_csv(improved, cost_mat, supply_bal, demand_bal, path="optimal_plan.csv")
    print("Saved: optimal_plan.csv")

# -------------------- CLI --------------------
def main():
    parser = argparse.ArgumentParser(description="Transport Suite: NWC / LCM / VAM (+ optional Stepping-Stone)")
    parser.add_argument("--method", type=str, default="vam", choices=["nwc", "lcm", "vam"], help="Initial method")
    parser.add_argument("--optimal", action="store_true", help="Run Stepping-Stone after the initial method")
    parser.add_argument("--quiet", action="store_true", help="Less verbose output")
    args = parser.parse_args()

    solve_transport(SUPPLY, DEMAND, COST, method=args.method, do_optimal=args.optimal, quiet=args.quiet)



## Example Run

# 3

In [31]:
# ====== EDIT YOUR DATA HERE ======
M = 1000
SUPPLY = [1200, 1400, 1600, 1200]
DEMAND = [1500, 800, 2000]
COST = [
    [5, 9, 12],
    [8, 10, 9],
    [5, 6, 8],
    [6, 8, 10]
]
# ถ้าจะเปลี่ยนเป็นโจทย์ "รายได้สูงสุด" (maximize profit) ให้ใช้ค่าลบของรายได้แทน cost:
# COST = [[-p for p in row] for row in COST]
# =================================

In [32]:
solve_transport(SUPPLY, DEMAND, COST, method='vam', do_optimal=True, quiet=False)

=== Vogel's Approximation Method (VAM) ===
Balanced problem:
  supply = [1200, 1400, 1600, 1200]
  demand = [1500, 800, 2000, 1100]
  added = {'dummy_row': False, 'dummy_col': True}

Step 1: penalties
  col[1] -> min1=5.0000, min2=5.0000, penalty=0.0000
  col[2] -> min1=6.0000, min2=8.0000, penalty=2.0000
  col[3] -> min1=8.0000, min2=9.0000, penalty=1.0000
  col[4] -> min1=0.0000, min2=0.0000, penalty=0.0000
  row[1] -> min1=0.0000, min2=5.0000, penalty=5.0000
  row[2] -> min1=0.0000, min2=8.0000, penalty=8.0000
  row[3] -> min1=0.0000, min2=5.0000, penalty=5.0000
  row[4] -> min1=0.0000, min2=6.0000, penalty=6.0000

  Chosen: row[2] with max penalty 8.0000
  Allocate 1100.0000 units to cell (2,4) with cost 0.0000
  Remaining supply[2]=300.0000, demand[4]=0.0000

Step 2: penalties
  col[1] -> min1=5.0000, min2=5.0000, penalty=0.0000
  col[2] -> min1=6.0000, min2=8.0000, penalty=2.0000
  col[3] -> min1=8.0000, min2=9.0000, penalty=1.0000
  row[1] -> min1=5.0000, min2=9.0000, penalty=4.

# 4

In [37]:
# ====== EDIT YOUR DATA HERE ======
M = 10000
SUPPLY = [50, 40, 50]
DEMAND = [30, 50, 30, 20]
COST = [
    [17, 13, 20, 17],
    [14, M, 19, 15],
    [15, 20, 23, 18],
]
# ถ้าจะเปลี่ยนเป็นโจทย์ "รายได้สูงสุด" (maximize profit) ให้ใช้ค่าลบของรายได้แทน cost:
# COST = [[-p for p in row] for row in COST]
# =================================

In [38]:
solve_transport(SUPPLY, DEMAND, COST, method='vam', do_optimal=False, quiet=False)

=== Vogel's Approximation Method (VAM) ===
Balanced problem:
  supply = [50, 40, 50]
  demand = [30, 50, 30, 20, 10]
  added = {'dummy_row': False, 'dummy_col': True}

Step 1: penalties
  col[1] -> min1=14.0000, min2=15.0000, penalty=1.0000
  col[2] -> min1=13.0000, min2=20.0000, penalty=7.0000
  col[3] -> min1=19.0000, min2=20.0000, penalty=1.0000
  col[4] -> min1=15.0000, min2=17.0000, penalty=2.0000
  col[5] -> min1=0.0000, min2=0.0000, penalty=0.0000
  row[1] -> min1=0.0000, min2=13.0000, penalty=13.0000
  row[2] -> min1=0.0000, min2=14.0000, penalty=14.0000
  row[3] -> min1=0.0000, min2=15.0000, penalty=15.0000

  Chosen: row[3] with max penalty 15.0000
  Allocate 10.0000 units to cell (3,5) with cost 0.0000
  Remaining supply[3]=40.0000, demand[5]=0.0000

Step 2: penalties
  col[1] -> min1=14.0000, min2=15.0000, penalty=1.0000
  col[2] -> min1=13.0000, min2=20.0000, penalty=7.0000
  col[3] -> min1=19.0000, min2=20.0000, penalty=1.0000
  col[4] -> min1=15.0000, min2=17.0000, penal

# 6


In [33]:
# ====== EDIT YOUR DATA HERE ======
M = 10000
SUPPLY = [3, 2, 2]
DEMAND = [1, 1, 1, 1, 1]
COST = [
    [8, 4, 6, 7, 9],
    [8, 5, 7, 7, 10],
    [9, 6, 5, 6, 9],
]
# ถ้าจะเปลี่ยนเป็นโจทย์ "รายได้สูงสุด" (maximize profit) ให้ใช้ค่าลบของรายได้แทน cost:
# COST = [[-p for p in row] for row in COST]
# =================================

In [35]:
solve_transport(SUPPLY, DEMAND, COST, method='vam', do_optimal=True, quiet=False)

=== Vogel's Approximation Method (VAM) ===
Balanced problem:
  supply = [3, 2, 2]
  demand = [1, 1, 1, 1, 1, 2]
  added = {'dummy_row': False, 'dummy_col': True}

Step 1: penalties
  col[1] -> min1=8.0000, min2=8.0000, penalty=0.0000
  col[2] -> min1=4.0000, min2=5.0000, penalty=1.0000
  col[3] -> min1=5.0000, min2=6.0000, penalty=1.0000
  col[4] -> min1=6.0000, min2=7.0000, penalty=1.0000
  col[5] -> min1=9.0000, min2=9.0000, penalty=0.0000
  col[6] -> min1=0.0000, min2=0.0000, penalty=0.0000
  row[1] -> min1=0.0000, min2=4.0000, penalty=4.0000
  row[2] -> min1=0.0000, min2=5.0000, penalty=5.0000
  row[3] -> min1=0.0000, min2=5.0000, penalty=5.0000

  Chosen: row[2] with max penalty 5.0000
  Allocate 2.0000 units to cell (2,6) with cost 0.0000
  Remaining supply[2]=0.0000, demand[6]=0.0000

Step 2: penalties
  col[1] -> min1=8.0000, min2=9.0000, penalty=1.0000
  col[2] -> min1=4.0000, min2=6.0000, penalty=2.0000
  col[3] -> min1=5.0000, min2=6.0000, penalty=1.0000
  col[4] -> min1=6.0